In [2]:
import sys
import getopt
import pandas as pd
import re
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError

In [8]:
import sys
import os
import getopt
import re
import unicodedata

import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError

def parse_arguments():
    """
    (Opcional) Se passado via CLI, captura um único arquivo CSV.
    """
    unix_opts = "f:"
    gnu_opts = ["file="]

    args, _ = getopt.getopt(sys.argv[1:], unix_opts, gnu_opts)
    for opt, val in args:
        if opt in ("-f", "--file"):
            return val
    return None


def extract_year_from_path(file_path):
    """
    Extrai o primeiro padrão de 4 dígitos (YYYY) encontrado no nome do arquivo.
    """
    name = os.path.basename(file_path)
    m = re.search(r"(\d{4})", name)
    if not m:
        raise ValueError(f"Ano não encontrado em: {name}")
    return int(m.group(1))


def create_db_engine(db_path):
    """
    Cria engine SQLite via SQLAlchemy.
    """
    url = f"sqlite:///{db_path}"
    engine = create_engine(url)
    print(f"[INFO] Conectado ao banco: {url}")
    return engine


def data_already_exists(engine, table_name, year):
    """
    Verifica se já existem registros para ANO_EGRESO = year.
    """
    try:
        with engine.connect() as conn:
            q = text(f"SELECT 1 FROM {table_name} WHERE ANO_EGRESO = :y LIMIT 1")
            return conn.execute(q, {"y": year}).fetchone() is not None
    except OperationalError:
        return False


def load_data(file_path):
    """
    Lê CSV delimitado por ponto-e-vírgula, codificação latin1.
    """
    df = pd.read_csv(file_path, encoding="latin1", delimiter=";")
    print("Colunas encontradas:", df.columns.tolist())
    return df


def normalize_col(name):
    """
    Normaliza nome de coluna: remove acentos, substitui espaços por '_', maiúsculas.
    """
    nf = unicodedata.normalize("NFKD", name)
    no_accent = "".join(ch for ch in nf if not unicodedata.combining(ch))
    return no_accent.replace(" ", "_").upper()


def preprocess_data(df, threshold=0.5):
    """
    - Normaliza cabeçalhos
    - Remove linhas vazias e com '*' em excesso
    - Converte colunas chave para inteiro
    - Renomeia colunas existentes conforme mapeamento final
    """
    # 1) normalizar nomes de coluna
    df.columns = [normalize_col(c) for c in df.columns]
    print("Colunas após normalização:", df.columns.tolist())

    # 2) remover linhas completamente vazias
    df = df.dropna(how="all")

    # 3) filtrar linhas com '*' em excesso
    num_cols = len(df.columns)
    allowed = int(num_cols * threshold)
    df = df[df.apply(lambda row: (row == "*").sum() <= allowed, axis=1)]

    # 4) converter colunas-chave para inteiro, se existirem
    for col in ["COMUNA_RESIDENCIA", "REGION_RESIDENCIA", "ANO_EGRESO"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
        else:
            print(f"[WARN] coluna '{col}' não encontrada, pulando conversão.")

    # 5) renomear para schema final, apenas colunas presentes
    final_mapping = {
        "PERTENENCIA_ESTABLECIMIENTO_SALUD": "PERTENENCIA_ESTABLECIMIENTO_SALUD",
        "SEXO": "SEXO",
        "GRUPO_EDAD": "GRUPO_EDAD",
        "ETNIA": "ETNIA",
        "GLOSA_PAIS_ORIGEN": "GLOSA_PAIS_ORIGEN",
        "COMUNA_RESIDENCIA": "COMUNA_RESIDENCIA",
        "GLOSA_COMUNA_RESIDENCIA": "GLOSA_COMUNA_RESIDENCIA",
        "REGION_RESIDENCIA": "REGION_RESIDENCIA",
        "GLOSA_REGION_RESIDENCIA": "GLOSA_REGION_RESIDENCIA",
        "PREVISION": "PREVISION",
        "GLOSA_PREVISION": "GLOSA_PREVISION",
        "ANO_EGRESO": "ANO_EGRESO",
        "DIAG1": "DIAG1",
        "DIAG2": "DIAG2",
        "DIAS_ESTADA": "DIAS_ESTADA",
        "CONDICION_EGRESO": "CONDICION_EGRESO",
        "INTERV_Q": "INTERV_Q",
        "PROCED": "PROCED"
    }
    # filtrar apenas chaves existentes
    mapping = {old: new for old, new in final_mapping.items() if old in df.columns}
    df = df.rename(columns=mapping)

    return df


def save_to_database(df, engine, table_name):
    """
    Insere (append) o DataFrame no banco de dados.
    """
    df.to_sql(table_name, engine, if_exists="append", index=False)
    print(f"[INFO] Inseridos {len(df)} registros em '{table_name}'.")


def validate_data(engine, table_name):
    """
    Exibe contagem de registros por ano.
    """
    with engine.connect() as conn:
        q = text(
            f"SELECT ANO_EGRESO, COUNT(*) AS total "
            f"FROM {table_name} GROUP BY ANO_EGRESO"
        )
        for ano, total in conn.execute(q):
            print(f"Ano {ano}: {total} registros")


if __name__ == "__main__":
    table_name = "egresos_pacientes"
    engine = create_db_engine("database/ministerio_de_salud_chile.db")

    # captura via CLI (opcional)
    single_file = parse_arguments()

    if single_file:
        files = [single_file]
    else:
        base_folder = r"C:\Users\jonat\Documents\GitHub\Sprints\Sprint 12\Automação\dados"
        files = [
            os.path.join(base_folder, "EGRE_DATOS_ABIERTOS_2018.csv"),
            os.path.join(base_folder, "EGRE_DATOS_ABIERTOS_2019.csv"),
            os.path.join(base_folder, "EGRE_DATOS_ABIERTOS_2020.csv"),
        ]

    for fp in files:
        print(f"\n[INFO] Processando: {fp}")
        year = extract_year_from_path(fp)

        if data_already_exists(engine, table_name, year):
            print(f"[SKIP] Dados de {year} já existem.")
            continue

        df = load_data(fp)
        df = preprocess_data(df)
        save_to_database(df, engine, table_name)

    print("\n[INFO] Validação final:")
    validate_data(engine, table_name)


[INFO] Conectado ao banco: sqlite:///database/ministerio_de_salud_chile.db

[INFO] Processando: c:\Users\jonat\AppData\Roaming\jupyter\runtime\kernel-v3a484d85a8593677eb75b234c4bbd7b98bb7a83b6.json
Colunas encontradas: ['{"key":"c967b2f8-f06d-44b6-ac5c-e3542a59a78a","signature_scheme":"hmac-sha256","transport":"tcp","ip":"127.0.0.1","hb_port":9005,"control_port":9006,"shell_port":9007,"stdin_port":9008,"iopub_port":9009,"kernel_name":"python3125jvsc74a57bd0de0d5ab311a6ab3d90707e4bf718a3da608e8516e1ae98c36f30890de738415a"}']
Colunas após normalização: ['{"KEY":"C967B2F8-F06D-44B6-AC5C-E3542A59A78A","SIGNATURE_SCHEME":"HMAC-SHA256","TRANSPORT":"TCP","IP":"127.0.0.1","HB_PORT":9005,"CONTROL_PORT":9006,"SHELL_PORT":9007,"STDIN_PORT":9008,"IOPUB_PORT":9009,"KERNEL_NAME":"PYTHON3125JVSC74A57BD0DE0D5AB311A6AB3D90707E4BF718A3DA608E8516E1AE98C36F30890DE738415A"}']
[WARN] coluna 'COMUNA_RESIDENCIA' não encontrada, pulando conversão.
[WARN] coluna 'REGION_RESIDENCIA' não encontrada, pulando conve

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [7]:
print(df.columns.tolist())

['{"key":"c967b2f8-f06d-44b6-ac5c-e3542a59a78a","signature_scheme":"hmac-sha256","transport":"tcp","ip":"127.0.0.1","hb_port":9005,"control_port":9006,"shell_port":9007,"stdin_port":9008,"iopub_port":9009,"kernel_name":"python3125jvsc74a57bd0de0d5ab311a6ab3d90707e4bf718a3da608e8516e1ae98c36f30890de738415a"}']
